In [13]:
# imports
from visualisation.vis_utils.read_csv_to_df import df_from_csv
import pandas as pd
import plotly.graph_objects as go
from IPython.display import Image
from visualisation.vis_chars_and_ships import (
    make_full_chars_df, 
    make_hottest_char_df, 
    visualise_hottest_characters
)

In [16]:
# setting up hottest chars df & fig

characters_df = df_from_csv("data/fifth_clean_up_data/stage_5_characters.csv")
ships_df = df_from_csv("data/fifth_clean_up_data/stage_5_ships.csv")

full_character_df = make_full_chars_df(characters_df, ships_df)
hottest_rank_df = make_hottest_char_df(full_character_df)
hottest_rank_fig = visualise_hottest_characters(hottest_rank_df)

# hottest_rank_fig.write_image(
#     "visualisation/all_ao3_data_vis_charts/all_ao3_hottest_characters_ranking.png", 
#     width=800, 
#     height=1400, 
#     scale=2
# )

hottest_rank_fig.show(
    width=800, 
    height=1400, 
    scale=2
) 
# non-kaleido one displays kana properly but won't let me export 
# the whole table as one picture, even when specifying dimensions smh
    # tried to just export the bit with the kana (to manually photoshop em together), 
    # but it defaults to just exporting the top bit smh

# img_bytes = hottest_rank_fig.to_image(
#     format="png", 
#     width=800, 
#     height=1400, 
#     scale=2
# )
# Image(img_bytes) 
# # this still uses kaleido & doesn't render non-latin characters smh